In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt 
from datetime import timedelta as delta
import statsmodels.api as sm
from scipy import stats, interpolate
from shapely.geometry import shape
from functools import reduce
import json

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.multiclass import OutputCodeClassifier
import xgboost as xgb
import lightgbm as lgbm
from sktime.classification.interval_based import SupervisedTimeSeriesForest, TimeSeriesForestClassifier

import torch
import torch.nn as nn

ModuleNotFoundError: No module named 'statsmodels'

In [2]:
from torchmetrics.classification import MulticlassRecall
from torch.optim.lr_scheduler import _LRScheduler
from scipy.signal import savgol_filter

In [134]:
from torch.nn import functional as F
from tqdm import tqdm

In [95]:
class NDVIDataset(torch.utils.data.Dataset):
    def __init__(self, df, mode = 'train'):
        self.mode = mode
        self.df = df.copy()
        self.df.sort_index(axis=1, inplace=True)
        self.df.set_index('id', inplace=True)
        
        self.target_col = pd.Index(['crop'])
        self.area_col = pd.Index(['area'])
        self.geo_col = pd.Index(['.geo'])
        self.ts_cols = self.df.columns.difference(self.area_col.append([self.geo_col, self.target_col])).to_list()
        
        if self.mode == 'train' or self.mode == 'valid':
            self.targets = self.df[self.target_col].copy()
            self.features = self.df[self.ts_cols].copy()
        elif self.mode == 'test':
            self.features = self.df[self.ts_cols].copy()

        self.first_date = dt.strptime("2021-04-15", "%Y-%m-%d")
        self.last_date = dt.strptime("2021-08-27", "%Y-%m-%d")
        self.n_days = (self.last_date - self.first_date).days + 1
        
        new_names = {old_name: (dt.strptime(old_name.split('_')[-1], "%Y-%m-%d") - self.first_date).days+1 for old_name in self.ts_cols}
        self.features.rename(new_names, axis=1, inplace=True)
        self.features.sort_index(axis=1, inplace=True)
        self.ts_cols = self.features.columns.to_list()
        
        self.all_dates = np.array(range(1, self.n_days + 1))
        self.existed_dates = np.array(self.ts_cols)
        self.new_dates = np.setdiff1d(self.all_dates, self.existed_dates)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        row = self.expand_dates(self.features.iloc[idx])
        row = np.clip(row, 0, 1)
        row = self.interpolate_(row)
        
        feature = torch.tensor(row.values, dtype=torch.float32)
        
        if self.mode == 'train' or self.mode == 'valid':
            target = torch.tensor(self.targets.iloc[idx].values, dtype=torch.int64)
            return {'features': feature,
                    'target': target}
        else:
            return {'features': feature}
        
    def n_cols(self):
        return self.n_days
            
    def expand_dates(self, row):
        new_row = pd.Series(index = self.all_dates, dtype=np.float64, name=row.name)
        new_row[self.existed_dates] = row[self.existed_dates].copy()
        new_row[self.new_dates] = np.zeros(self.new_dates.shape[0])
        new_row = new_row.astype(np.float32)

        return new_row
    
    def local_max_interpolation(self, row, window_s):
        reconstructed_max = pd.Series(index = row.index, dtype = np.float64)

        for i in row.index:
            start = max(1, i - window_s//2)
            end = min(window_s//2 + 1 + start, row.index[-1]) + 1

            max_window = np.intersect1d(np.array(range(start, end)), row.index)
            reconstructed_max[i] = row.loc[max_window].max()

        return reconstructed_max
    
    def interpolate_(self, row):
        values = savgol_filter(self.local_max_interpolation(row, 10), 10, 2)
        return pd.Series(values, index = row.index, dtype=np.float64)

In [133]:
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.bidirectional = False
        
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=self.bidirectional)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        x = x.view(x.size(0), x.size(1), self.input_dim)
        
        self.rnn.flatten_parameters()
        out, _ = self.rnn(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        d = 2 if self.bidirectional == True else 1
        h0 = torch.zeros(d * self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(d * self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [5]:
class CyclicLR(_LRScheduler):
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]
    
def cosine(t_max, eta_min=0):
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [135]:
class Agent:
    def __init__(self, model, device, optimizer=None, criterion=None, sheduler=None):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = sheduler
        
        if torch.cuda.device_count() > 1:
            self.model = nn.DataParallel(self.model)
        self.model.to(self.device)

        self.ckpt_path = "models"
        if not os.path.exists(self.ckpt_path):
            os.mkdir(self.ckpt_path)
        
        self.metric = MulticlassRecall(7, average='macro').to(self.device)
        
    def train(self, n_epochs, train_loader, valid_loader):
        best_recall = 0
        patience = 100
        trials = 0
        
        for epoch in range(1, n_epochs + 1):
            self.model.train()
            pbar = tqdm(train_loader)
            for i, batch in enumerate(pbar):
                features = batch['features'].to(self.device).float()
                target = batch['target'].to(self.device).long()[:,0]
                
                self.optimizer.zero_grad()
                
                prediction = self.model(features)     
                loss = self.criterion(prediction, target)
                
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                
                pbar.set_description("EPOCH[{}/{}]".format(epoch, n_epochs))
                pbar.set_postfix(loss=loss.item())

            self.model.eval()
            pbar = tqdm(valid_loader, colour='green')
            
            general_target = torch.ones((1,1)).to(self.device)
            general_prediction = torch.ones((1,1)).to(self.device)
            
            with torch.no_grad():
                for i, batch in enumerate(pbar):
                    features = batch['features'].to(self.device).float()
                    target = batch['target'].to(self.device).long()[:,0]
                    prediction = self.model(features)
                    prediction = F.log_softmax(prediction, dim=1).argmax(dim=1)
                    
                    target = target.reshape(1,-1)
                    prediction = prediction.reshape(1,-1)
                    
                    general_target = torch.concat([general_target, target], axis=1)
                    general_prediction = torch.concat([general_prediction, prediction], axis=1)
                    recall = self.metric(general_prediction[:,1:], general_target[:,1:]).item()

                    pbar.set_description("EVALUATION")
                    pbar.set_postfix(Recall = recall)

            if epoch % 5 == 0:
                print(f'Epoch: {epoch:3d}. Acc.: {recall:2.2%}')

            if recall > best_recall:
                trials = 0
                best_recall = recall
                torch.save(model.state_dict(), 'models/best.pth')
                print(f'Epoch {epoch} best model saved with recall: {best_recall:2.2%}')
            else:
                trials += 1
                if trials >= patience:
                    print(f'Early stopping on epoch {epoch}')
                    break

In [2]:
def local_max_interpolation(row, window_s):
    reconstructed_max = pd.Series(index = row.index, dtype = np.float64)

    for i in row.index:
        start = max(1, i - window_s//2)
        end = min(window_s//2 + 1 + start, row.index[-1]) + 1
        
        max_window = np.intersect1d(np.array(range(start, end)), row.index)
        reconstructed_max[i] = row.loc[max_window].max()
        
    return reconstructed_max

def preprocess(data, 
               train=True,   
               rename=False, 
               expand_dates=False,
               scaling=False, 
               boxcox=False, 
               diff=False):
    
    def parse_json(obj):
        coords = shape(json.loads(obj.values[0])).bounds
        x = (coords[0] + coords[2]) / 2
        y = (coords[1] + coords[3]) / 2
        return (x, y)
    
    def interpolate_(row):
        values = savgol_filter(local_max_interpolation(row, 10), 10, 2)
        return pd.Series(values, index = row.index, dtype=np.float64)
    
    def fill_fl_nan(row):
        not_nans = np.where(np.logical_not(row.isna()))[0]
        if not_nans[0] > 0:
            row.iloc[0] = row.iloc[not_nans[0]]
        if not_nans[-1] < row.shape[0]:
            row.iloc[-1] = row.iloc[not_nans[-1]]

        return row
    
    def rename_cols(df, cols_to_rename):
        new_names = {old_name: (dt.strptime(old_name.split('_')[-1], "%Y-%m-%d") - first_date).days+1 for old_name in cols_to_rename}
        df = df.rename(new_names, axis=1)
        
        return df

    def expand_dates_(row):
        new_row = pd.Series(index = all_dates, dtype=np.float64, name=row.name)
        new_row[existed_dates] = row[existed_dates].copy()
        new_row[new_dates] = np.zeros(new_dates.shape[0])
        new_row = new_row.astype(np.float32)

        return new_row

    target_col = pd.Index(['crop'])
    area_col = pd.Index(['area'])
    geo_col = pd.Index(['.geo'])
    ts_cols = data.columns.difference(area_col.append([geo_col, target_col])).to_list()
    
    # Mode
    if train:
        target = data[target_col]
        features = data.drop(target_col.to_list(), axis=1)
    else:
        target = None
        features = data
        
    first_date = dt.strptime("2021-04-15", "%Y-%m-%d")
    last_date = dt.strptime("2021-08-27", "%Y-%m-%d")
    n_days = (last_date - first_date).days + 1

    all_dates = np.array(range(1, n_days + 1))
    existed_dates = np.array(ts_cols)
    new_dates = np.setdiff1d(all_dates, existed_dates)
    
    # Renaiming
    if rename:
        features = rename_cols(features, ts_cols)
        ts_cols = features.columns.difference(area_col.append([geo_col, target_col])).to_list()
        
        existed_dates = np.array(ts_cols)
        new_dates = np.setdiff1d(all_dates, existed_dates)
        
    features_ts = features[ts_cols].copy()
    
    # Expanding dates
    if expand_dates:
        features_ts = features_ts.apply(expand_dates_, axis=1)
        ts_cols = features_ts.columns.to_list()
        
    # Interpolation
    # features_ts[features_ts <= 0] = np.nan
    # features_ts[features_ts >= 1] = 1
    
#     features_ts = np.clip(features_ts, 0, 1)
#     features_ts.sort_index(axis=1, inplace=True)
    
    # features_ts = features_ts.apply(fill_fl_nan, axis=1)
#     features_ts = features_ts.apply(interpolate_, axis=1)
    
#     features_ts.interpolate(axis=1, inplace=True)
#     features_ts.ffill(axis=1, inplace=True)
#     features_ts.bfill(axis=1, inplace=True)
    
    # Scaling
    if scaling:
        global_min = features_ts.min()
        global_max = features_ts.max()

        features_ts = (features_ts - global_min)/(global_max - global_min)
    
    # BoxCox
    if boxcox:
        features_ts = features_ts.apply(lambda row: stats.boxcox(row)[0], axis=1)
        features_ts =  pd.DataFrame(features_ts.to_list(), columns=ts_cols, index=features_ts.index)
    
    # Diff
    if diff:
        features_ts = features_ts.apply(lambda row: row-row.shift(1), axis=1)
        to_remove = ts_cols[0:1]
        for el in to_remove:
            features_ts.drop([el], inplace=True, axis=1)
            ts_cols.remove(el)
    
    features_geo = features[geo_col].copy()
    coordinates = features_geo.apply(parse_json, axis=1)
    features_geo =  pd.DataFrame(coordinates.to_list(), columns=['x', 'y'], index=features_geo.index)
    
    features_area = features[area_col].copy()
    features_parts = [features_ts, features_geo, features_area]

    features = reduce(lambda left, right: left.join(right), features_parts)
    return features, target

def to_sktime_format(features):
    sktime_df = pd.DataFrame(columns=['ndvi'])
    for i, row in features.iterrows():        
        sktime_df.loc[i] = {'ndvi': row}
    
    return sktime_df

In [3]:
train_file = 'data/train_dataset_train.csv'
test_file = 'data/test_dataset_test.csv'

data = pd.read_csv(train_file)
data.sort_index(axis=1, inplace=True)
data.set_index('id', inplace=True)
features, target = preprocess(data, rename=True)
features_cols = features.columns

data = features.join(target)
data = shuffle(data)
features = data[features_cols]
target = data.crop

features.head()

NameError: name 'shape' is not defined

# Train

In [14]:
train_X, valid_X, train_Y, valid_Y = train_test_split(features, target, test_size=0.2, shuffle=True)

In [75]:
ts_forest_clf = SupervisedTimeSeriesForest()
boosting_clf = xgb.XGBClassifier()

In [87]:
# train_ts_proba = ts_forest_clf.fit_predict_proba(train_X[ts_cols].values, train_Y.values.ravel())
# valid_ts_proba = ts_forest_clf.predict_proba(valid_X[ts_cols].values)

In [88]:
# train_ts_proba = pd.DataFrame(train_ts_proba)
# train_ts_proba['id'] = train_Y.index
# train_ts_proba.set_index('id', inplace=True)

# train_X = train_X.join(train_ts_proba)

In [ ]:
# xgb_grid.fit(train_X, train_Y)

In [23]:
best_params = {'colsample_bytree': 0.5, 'eta': 0.05, 'gamma': 0.3, 'max_depth': 5, 'min_child_weight': 3}

In [22]:
boosting_clf = xgb.XGBClassifier(**best_params, n_estimators=1000)
scoring = lambda estimator, x, y: recall_score(y, estimator.predict(x), average="macro", zero_division=0)
cross_val_score(boosting_clf, features, target, cv=3, scoring=scoring)
# boosting_clf.fit(train_X, train_Y)
# pred_Y = boosting_clf.predict(valid_X)

array([0.97075529, 0.96718476, 0.97067481])

In [5]:
lgmb_clf = lgbm.LGBMClassifier(n_estimators=1000)
scoring = lambda estimator, x, y: recall_score(y, estimator.predict(x), average="macro", zero_division=0)
cross_val_score(lgmb_clf, features, target, cv=5, scoring=scoring)

array([0.97346194, 0.97461239, 0.96437764, 0.96732883, 0.97273371])

In [26]:
skf = StratifiedKFold(n_splits = 4)
skf.get_n_splits(features.values, target)
clf = OutputCodeClassifier(xgb.XGBClassifier(**best_params, n_estimators=100), code_size=4)
scoring = lambda estimator, x, y: recall_score(y, estimator.predict(x), average="macro", zero_division=0)
cross_val_score(clf, features.values, target.values, cv=skf, scoring=scoring)

array([0.96242078, 0.96431584, 0.96484257, 0.96052407])

In [ ]:
parameters = {
    'boosting':         ['gbdt', 'dart', 'goss', 'rf'],
    'num_iterations':   [1000, 1500, 2000, 5000],
    'learning_rate':    [0.05, 0.005, 0.01, 0.1, 0.001],
    'num_leaves':       [7, 15, 31, 42],
    'max_depth' :       [3, 5, 7, 9, 10, 15, 25],
    'min_data_in_leaf': [15, 20, 25],
    'feature_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_freq':     [100, 300, 200, 400, 500],
    'reg_alpha':        [0, 0.5, 0.7],
    'reg_lambda':       [0, 0.5, 0.7],
 
}

In [ ]:
lgbm.LGBMClassifier(n_estimators=1000)

In [91]:
print(f'Recall: {recall_score(valid_Y, pred_Y, average="macro", zero_division=0)}')

Recall: 0.9300789100701147


In [52]:
print('Boosting')
print('Recall: 0.9375329486986167')
print('Boosting eta=0.15 gamma=0.1')
print('Recall: 0.9407863121873382')
print('Boosting boxcox by day')
print('Recall: 0.9473720633651918')
print('Boosting boxcox')
print('Recall: 0.9550711109767024')
print('Boosting boxcox')
print('Recall: 0.9550711109767024')

Boosting
Recall: 0.9375329486986167
Boosting eta=0.15 gamma=0.1
Recall: 0.9407863121873382
Boosting boxcox by day
Recall: 0.9473720633651918
Boosting boxcox
Recall: 0.9550711109767024


# Test

In [27]:
test_data = pd.read_csv(test_file)
test_data.sort_index(axis=1, inplace=True)
test_data.set_index('id', inplace=True)
test_data.head()

,.geo,area,nd_mean_2021-04-15,nd_mean_2021-04-16,nd_mean_2021-04-18,nd_mean_2021-04-19,nd_mean_2021-04-20,nd_mean_2021-04-22,nd_mean_2021-04-23,nd_mean_2021-04-25,...,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-01,nd_mean_2021-08-07,nd_mean_2021-08-10,nd_mean_2021-08-11,nd_mean_2021-08-12,nd_mean_2021-08-13,nd_mean_2021-08-23,nd_mean_2021-08-27
id,,,,,,,,,,,,,,,,,,,,,
611,"{""type"":""Polygon"",""coordinates"":[[[35.96545926...",26,0.083038,0.196691,0.244827,0.000000,0.241219,0.259303,0.064947,0.066446,...,0.719171,0.000000,0.593911,0.000000,0.000000,0.192016,0.164840,0.714292,0.065030,0.084197
6417,"{""type"":""Polygon"",""coordinates"":[[[39.50520518...",98,0.122470,0.091192,0.241003,0.000000,0.181600,0.125823,0.108175,0.081871,...,0.317670,0.329803,0.000000,0.007428,0.000000,0.106484,0.167479,0.000000,0.000000,0.108945
3352,"{""type"":""Polygon"",""coordinates"":[[[39.30105409...",21,0.259347,0.000000,0.297489,0.033368,0.000000,0.285057,0.374026,0.000000,...,0.534127,0.000000,0.549128,0.000000,0.026141,0.469352,0.477381,0.037838,0.000000,0.158279
4224,"{""type"":""Polygon"",""coordinates"":[[[33.08144648...",18,0.166209,0.111243,0.000000,0.201012,0.083885,0.000000,0.015033,0.065305,...,0.765292,0.597874,0.681923,0.035716,0.693845,0.614671,0.634286,0.108990,0.532947,0.237742
3102,"{""type"":""Polygon"",""coordinates"":[[[45.10231339...",53,0.010862,0.000000,0.067479,0.234954,0.000000,0.000000,0.000000,0.103583,...,0.000000,0.134136,0.780305,0.030224,0.046893,0.000000,0.779191,0.048622,0.650742,0.121296


In [28]:
test_features, _ = preprocess(test_data, train=False, rename=True)

In [30]:
# lgmb_clf = lgbm.LGBMClassifier(n_estimators=1000)
# lgmb_clf.fit(features, target)
clf = OutputCodeClassifier(xgb.XGBClassifier(**best_params, n_estimators=1000), code_size=4)
clf.fit(features.values, target.values)

test_target = clf.predict(test_features.values)

In [31]:
result = pd.DataFrame(columns=['id', 'crop'])
result['id'] = test_features.index
result['crop'] = test_target

In [32]:
result.to_csv('results_v2.2.csv', index=False)

# RNN

In [13]:
train_file = 'data/train_dataset_train.csv'
test_file = 'data/test_dataset_test.csv'

In [98]:
train_df = pd.read_csv(train_file)
train_df, valid_df = train_test_split(train_df, test_size=0.2, shuffle=True)

In [105]:
train_dataset = NDVIDataset(train_df, 'train')
valid_dataset = NDVIDataset(valid_df, 'valid')

In [106]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           shuffle=True, 
                                           batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=batch_size)

In [136]:
input_dim = 1   
hidden_dim = 256
layer_dim = 3
output_dim = 7
# seq_dim = 128

lr = 0.0005
n_epochs = 1000
iterations_per_epoch = len(train_loader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleRNN(input_dim, hidden_dim, layer_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
sheduler = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

agent = Agent(model, device, optimizer, criterion, sheduler)

In [ ]:
agent.train(n_epochs, train_loader, valid_loader)

EVALUATION: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:58<00:00,  1.89s/it, Recall=0.357]


Epoch 1 best model saved with recall: 35.69%


EVALUATION: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:58<00:00,  1.90s/it, Recall=0.449]


Epoch 2 best model saved with recall: 44.92%


EVALUATION: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [01:00<00:00,  1.94s/it, Recall=0.556]


...

EVALUATION: 100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:58<00:00,  1.87s/it, Recall=0.749]


Epoch 28 best model saved with recall: 74.93%


EVALUATION: 100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:59<00:00,  1.91s/it, Recall=0.75]


Epoch:  30. Acc.: 75.03%
Epoch 30 best model saved with recall: 75.03%


EPOCH[31/1000]:   7%|█████▉                                                                         | 9/121 [00:17<03:41,  1.98s/it, loss=0.588]

In [ ]:
GridSearchCV()